<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/ocorrencias_aeronauticas_da_aviacao_civil_brasileira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img alt="Colaboratory logo" width="10%" src="https://github.com/leonarddocchagas/data_science/blob/master/projetos/CENIPA/logo_cenipa.png?raw=true" align="center">

## Análise dos Dados do CENIPA

#### **Centro de Investigação e Prevenção de Acidentes Aeronáuticos**

Mesmo se tratando do "transporte mais seguro", quem nunca sentiu 'frio na barriga' quando passa por uma turbuência ou mesmo simplesmente, quando sabe que precisa pegar um avião. 

Vamos investigar em busca de respostas sobre as causas dos acidentes e tentar descobrir se os desastres foram causados por falha humana ou falha da máquina.




In [1]:
# bibliotecas
import pandas as pd

In [16]:
# carrega e exibe dataset
df = pd.read_excel('/content/drive/My Drive/Colab Data Science/Projetos/CENIPA/ocorrencia_aviacao2.xlsx')
df.head()

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,39115,39115,39115,39115,39115,ACIDENTE,-133805555556,-446172222222,CORRENTINA,BA,BRASIL,SNTY,2010-02-07,17:40:00,SIM,FINALIZADA,A-031/CENIPA/2011,SIM,2011-07-21,2,1,NÃO
1,39155,39155,39155,39155,39155,INCIDENTE,NaN,NaN,BELO HORIZONTE,MG,BRASIL,****,2010-02-05,12:55:00,SIM,FINALIZADA,NaN,NÃO,NaT,0,1,NÃO
2,39156,39156,39156,39156,39156,INCIDENTE GRAVE,-152402777778,-593541666667,CASCAVEL,PR,BRASIL,SBCA,2010-01-10,23:15:00,SIM,FINALIZADA,I-004/CENIPA/2011,SIM,2011-06-30,2,1,NÃO
3,39158,39158,39158,39158,39158,INCIDENTE,***,***,BELÉM,PA,BRASIL,****,2010-01-28,16:00:00,SIM,FINALIZADA,NaN,NÃO,NaT,0,1,NÃO
4,39176,39176,39176,39176,39176,INCIDENTE,NaN,NaN,SÃO LUÍS,MA,BRASIL,****,2010-02-05,09:30:00,SIM,FINALIZADA,NaN,NÃO,NaT,0,1,NÃO


In [13]:
# carrega dataset e exibi os dados
df = pd.read_csv(
    'https://raw.githubusercontent.com/leonarddocchagas/data_science/master/projetos/CENIPA/ocorrencia_aviacao.csv', 
    error_bad_lines=False,
    sep=';',
    encoding = 'utf8'
    )
df

UnicodeDecodeError: ignored